In [ ]:
# https://bcm-uga.github.io/lfmm/reference/effect_size.html
# https://rdrr.io/github/bcm-uga/lfmm/man/effect_size.html

In [1]:
.libPaths()

[1] "/home/tbellagio/miniforge3/envs/pipeline_snakemake/lib/R/library"

In [2]:
.libPaths(c("/home/tbellagio/miniforge3/envs/r-environment/lib/R/library"))

In [3]:
library(lfmm)
library('data.table')

Warning message:
“package ‘lfmm’ was built under R version 4.3.2”
Warning message:
“package ‘data.table’ was built under R version 4.3.2”


In [4]:
split_number <- 0

In [5]:
path_ldp_af = '/carnegie/nobackup/scratch/xwu/grenet/merged_frequency/merged_hapFIRE_allele_frequency_LDpruned.txt'
results_dir = sprintf('/carnegie/nobackup/scratch/tbellagio/gea_grene-net/leave_1_out/split_number_%d', split_number)

k <- as.integer(readLines(sprintf('%s/num_components.txt', results_dir)))


file_name <- basename(path_ldp_af)
output_file_pvalue <- paste0(results_dir, "/pvalues.csv")
output_file_zscores <- paste0(results_dir, "/zscores.csv")
output_file_gif <- paste0(results_dir, "/gif.csv")
output_file_qvalue = paste0(results_dir, "/qvalues.csv")


allele_freq <- read.table(path_ldp_af, sep = '\t', header = TRUE, check.names = FALSE)


In [6]:
env_og <- read.csv(sprintf('%s/environment_lea.csv', results_dir), header = TRUE, colClasses = "numeric")

In [7]:
env <- read.csv(sprintf('%s/environment_lea.csv', results_dir), header = TRUE, colClasses = "numeric")
bio1_train = env[, 'bio1']
bio1_train <- as.matrix(bio1_train)

bio12_train = env[, 'bio12']
bio12_train <- as.matrix(bio12_train)


In [8]:
dim(env)

[1] 287   2

In [11]:
allele_freq <- allele_freq[, train_samples_vector]  # Use dynamic columns
colnames(allele_freq) <- NULL
allele_freq <- t(as.matrix(allele_freq))

In [13]:
env_test <- read.csv(sprintf('%s/testing_environment_lea_full_genome.csv', results_dir), header = TRUE, colClasses = "numeric")
bio1 = env_test[, 'bio1']
bio1 <- as.matrix(bio1)

bio12 = env_test[, 'bio12']
bio12 <- as.matrix(bio12)

In [14]:
## save the mean to then reescale 
af_means <- colMeans(allele_freq)

In [16]:
dim(matrix(af_means))

[1] 13985     1

In [17]:
effect_size

function (Y, X, lfmm.object) 
{
    if (ncol(X) > 1) 
        stop("Indirect effect sizes are computed for \n                        a single variable (d=1).")
    Y = scale(Y, scale = FALSE)
    X = scale(X, scale = FALSE)
    reg.lm <- function(i) {
        datafr <- data.frame(Y[, i], lfmm.object$U)
        lm(X ~ ., data = datafr)$coefficients[2]
    }
    p <- ncol(Y)
    effect.sizes <- sapply(1:p, FUN = reg.lm)
    return(effect.sizes)
}
<bytecode: 0x559fd5fa7460>
<environment: namespace:lfmm>

In [15]:
k=3

In [16]:
## redefine the nvironemtn as just the first vio1
env = matrix(env[,1])

In [147]:
env <- read.csv(sprintf('%s/environment_lea_full_genome.csv', results_dir), header = TRUE, colClasses = "numeric")

rownames(env) <- NULL
colnames(env) <- NULL
env <- as.matrix(env)

In [19]:
## Fit an LFMM with K = 6 factors
mod.lfmm <- lfmm_ridge(Y = allele_freq,
                       X = env,
                       K = k)

## Perform association testing using the fitted model:
pv <- lfmm_test(Y = allele_freq,
                X = env,
                lfmm = mod.lfmm,
                calibrate = "gif")

In [26]:
uv <- mod.lfmm$U %*% t(mod.lfmm$V)

In [27]:
head(uv)

-0.04017655,-0.096358955,-0.03341406,-0.062859896,-0.07706233,-0.008404517,0.009455899,-0.04143418,0.0476733,0.119051083,⋯,0.13872999,0.01354632,-0.06421124,-0.06148347,-0.03870015,0.03074705,-0.03239690,0.04436507,-0.0017865573,-0.0094456425
-0.03121768,0.024623859,-0.01390809,-0.009692986,-0.10489385,-0.042530520,0.025630907,0.14254598,0.1909064,0.005050331,⋯,0.08922710,-0.01540565,-0.08883609,-0.04960102,-0.04407516,0.21165041,-0.01280389,-0.05411738,-0.0001786726,0.0073490038
-0.01804376,0.249490226,-0.06177620,0.277136691,-0.25719531,-0.038162525,-0.042736713,0.18342337,0.2568694,0.312037434,⋯,-0.27538373,0.03292063,-0.11899236,-0.12901298,-0.07234014,-0.27462501,-0.07980592,0.05549793,-0.0692568495,-0.0615046505
-0.03471389,0.200973773,-0.08076844,0.136332591,-0.35857666,0.011840665,-0.048136808,0.31851025,0.3371333,0.276296487,⋯,0.18940965,-0.07159008,-0.07682143,-0.10359825,-0.01903587,0.33687400,-0.05042521,-0.06758046,-0.1026674569,-0.0159096425
-0.02325326,0.045173126,0.06631660,0.146931708,0.10869930,-0.024144954,0.003238304,0.03911233,0.1373654,0.105803084,⋯,0.01048224,-0.05259743,-0.05879883,0.01372184,-0.01758831,-0.14574886,-0.04210012,0.10486760,-0.0409785199,-0.0489348745
-0.04418243,0.001552687,-0.08035438,0.256890556,0.08744361,-0.052275032,-0.048425438,-0.15618538,0.1763458,0.295570193,⋯,0.34462116,-0.03200985,-0.11609519,-0.14397288,-0.05617695,-0.19072528,0.23449354,0.11546979,-0.0665848431,0.0001719299


In [23]:
dim(mod.lfmm$V)

[1] 13985    69

In [22]:
dim(mod.lfmm$U)

[1] 287  69

In [156]:
bio1_beta = matrix(pv$B[,1])
bio12_beta = matrix(pv$B[,2])

In [159]:
bio1_pvalues = matrix(pv$pvalue[,1])
bio12_pvalues = matrix(pv$pvalue[,2])

In [94]:
indices <- which(pv$pvalue[,1] < 1.5453657571674064e-08)

In [95]:
indices

[1]  1865 12078 12805

In [96]:
beta_i = pv$B

In [99]:
#beta_i = matrix(beta_i[, 1])

In [100]:
dim(env) # 2871

dim(beta_i) # 139851

[1] 287   1

[1] 13985     1

In [141]:
pred_af = bio1 %*% t(beta_i) + rep(af_means, each = nrow(bio1))

In [123]:
## adding the mean allele freq as an intercept
## in r thisis how you do it 
pred_af = env %*% t(beta_i) + rep(af_means, each = nrow(env))

In [161]:
write.csv(indices, file = 'here')


In [124]:
pred_af_filt = pred_af[,indices]

In [125]:
allele_freq_filt = allele_freq[,indices]

In [126]:
write.csv(allele_freq_filt, 'af.csv')

In [127]:
correlations <- sapply(1:ncol(allele_freq_filt), function(i) cor(allele_freq_filt[, i],pred_af_filt[, i]))

In [128]:
correlations

[1] 0.3932055 0.3019925 0.2371121

In [88]:
summary(pred_af_filt)

       V1                V2               V3        
 Min.   :0.02279   Min.   :0.3602   Min.   :0.1101  
 1st Qu.:0.06952   1st Qu.:0.4382   1st Qu.:0.1658  
 Median :0.08826   Median :0.4695   Median :0.1881  
 Mean   :0.10052   Mean   :0.4899   Mean   :0.2027  
 3rd Qu.:0.13344   3rd Qu.:0.5449   3rd Qu.:0.2420  
 Max.   :0.18814   Max.   :0.6362   Max.   :0.3072  

In [89]:
summary(allele_freq_filt)

       V1                V2                 V3        
 Min.   :0.00000   Min.   :0.001665   Min.   :0.0000  
 1st Qu.:0.01171   1st Qu.:0.376644   1st Qu.:0.1009  
 Median :0.05110   Median :0.477610   Median :0.1800  
 Mean   :0.10052   Mean   :0.489927   Mean   :0.2027  
 3rd Qu.:0.12434   3rd Qu.:0.599970   3rd Qu.:0.2682  
 Max.   :0.98923   Max.   :0.997608   Max.   :0.9959  

In [90]:
write.csv(pred_af_filt, 'firstmethod.csv')

In [32]:
#Y = allele_freq[,indices]

In [67]:

# Scale Y (centering only) and X (centering only)
Y <- scale(allele_freq, scale = FALSE)
#X <- scale(X, scale = FALSE)

# Regression function to get both intercept and slope for each SNP
reg.lm <- function(i) {
    datafr <- data.frame(env, mod.lfmm$U)  # Ensure this data frame is correctly formed
    model <- lm(Y[, i] ~ ., data = datafr)
    c(model$coefficients[1], model$coefficients[2])  # Extracting both intercept and slope
}

# Number of SNPs
p <- ncol(Y)

# Calculate effect sizes and intercepts using sapply
results <- t(sapply(1:p, FUN = reg.lm))  # Transpose to have SNPs in rows and coefficients in columns



In [68]:
filt_results = results[indices,]

In [69]:
filt_results

(Intercept),env
3.421007e-18,0.04617344
-1.679295e-18,0.07705458
1.198340e-17,0.05503610


In [70]:
pred_af2 = filt_results[, "(Intercept)"] + env %*% t(filt_results[, "env"])

In [71]:
write.csv(pred_af2, 'secondmethod.csv')

In [72]:
pred_af2

-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089
-0.03782975,-0.06313056,-0.04509089


In [42]:
correlations <- sapply(1:ncol(allele_freq_filt), function(i) cor(allele_freq_filt[, i],pred_af2[, i]))

In [43]:
correlations

[1] 0.3932055 0.3019925 0.2371121

In [556]:
train_samples_vector

[1] "1_2_2"   "1_2_3"   "1_3_1"   "1_3_4"   "1_3_5"   "1_3_6"   "1_3_7"  
  [8] "1_3_8"   "1_3_9"   "1_3_10"  "1_3_11"  "1_3_12"  "2_1_8"   "2_1_9"  
 [15] "2_1_11"  "2_1_12"  "2_2_5"   "2_2_6"   "2_2_7"   "5_3_1"   "5_3_2"  
 [22] "5_3_3"   "5_3_4"   "5_3_5"   "5_3_6"   "5_3_7"   "5_3_8"   "5_3_9"  
 [29] "5_3_10"  "5_3_11"  "5_3_12"  "6_1_1"   "6_1_4"   "6_2_2"   "6_2_3"  
 [36] "6_2_5"   "6_2_6"   "6_2_7"   "6_2_8"   "6_2_9"   "6_2_10"  "6_2_11" 
 [43] "6_2_12"  "9_3_1"   "9_3_2"   "9_3_3"   "9_3_4"   "9_3_5"   "9_3_6"  
 [50] "9_3_7"   "9_3_8"   "9_3_9"   "9_3_10"  "9_3_11"  "9_3_12"  "10_1_3" 
 [57] "10_1_4"  "10_1_6"  "10_1_7"  "10_1_11" "10_1_12" "10_2_1"  "10_2_5" 
 [64] "10_2_8"  "10_2_9"  "10_3_2"  "10_3_10" "11_1_4"  "11_1_8"  "11_1_9" 
 [71] "11_1_11" "11_2_2"  "11_2_7"  "11_2_12" "11_3_1"  "11_3_3"  "11_3_5" 
 [78] "11_3_6"  "12_1_2"  "12_1_3"  "12_1_5"  "12_1_6"  "12_1_7"  "12_1_8" 
 [85] "12_1_9"  "12_2_4"  "12_2_10" "12_2_11" "12_2_12" "13_1_6"  "13_2_7" 
 [92] "13_3_1"  "13_3_2"  "13_3_3"  "13_3_4"  "13_3_5"  "13_3_8"  "13_3_9" 
 [99] "13_3_10" "13_3_11" "13_3_12" "23_1_1"  "23_1_2"  "23_1_3"  "23_1_4" 
[106] "23_1_5"  "23_1_6"  "23_1_7"  "23_1_8"  "23_1_9"  "23_1_10" "23_1_11"
[113] "23_1_12" "25_2_1"  "25_2_2"  "25_2_3"  "25_2_4"  "25_2_5"  "25_2_6" 
[120] "25_2_7"  "25_2_8"  "25_2_9"  "25_2_10" "25_2_11" "25_2_12" "26_1_1" 
[127] "26_1_2"  "26_1_3"  "26_1_4"  "26_1_5"  "26_1_6"  "26_1_8"  "26_1_9" 
[134] "26_1_10" "26_1_11" "26_1_12" "27_2_2"  "27_2_3"  "27_2_5"  "27_3_1" 
[141] "27_3_4"  "27_3_6"  "27_3_7"  "27_3_8"  "27_3_9"  "27_3_10" "27_3_11"
[148] "27_3_12" "28_2_7"  "28_2_8"  "28_2_11" "28_3_2"  "28_3_3"  "28_3_4" 
[155] "28_3_6"  "28_3_12" "28_3_13" "28_3_15" "28_3_16" "28_3_17" "32_1_5" 
[162] "32_1_11" "32_2_8"  "32_2_9"  "32_3_1"  "32_3_2"  "32_3_3"  "32_3_4" 
[169] "32_3_6"  "32_3_7"  "32_3_10" "32_3_12" "33_1_1"  "33_2_4"  "33_2_5" 
[176] "33_2_6"  "33_2_9"  "33_2_11" "33_2_12" "37_1_1"  "37_1_2"  "37_1_3" 
[183] "37_1_4"  "37_1_5"  "37_1_6"  "37_1_7"  "37_1_8"  "37_1_9"  "37_1_10"
[190] "37_1_11" "37_1_12" "42_1_2"  "42_1_12" "42_2_11" "42_3_1"  "42_3_3" 
[197] "42_3_4"  "42_3_5"  "42_3_6"  "42_3_7"  "42_3_8"  "42_3_9"  "42_3_10"
[204] "43_1_3"  "43_1_10" "43_2_1"  "43_3_2"  "43_3_4"  "43_3_5"  "43_3_6" 
[211] "43_3_7"  "43_3_8"  "43_3_9"  "43_3_11" "43_3_12" "45_2_3"  "45_2_7" 
[218] "45_3_1"  "45_3_2"  "45_3_4"  "45_3_5"  "45_3_6"  "45_3_8"  "45_3_9" 
[225] "45_3_10" "45_3_11" "45_3_12" "46_3_1"  "46_3_2"  "46_3_3"  "46_3_4" 
[232] "46_3_5"  "46_3_6"  "46_3_7"  "46_3_8"  "46_3_9"  "46_3_10" "46_3_11"
[239] "46_3_12" "49_2_1"  "49_2_2"  "49_2_3"  "49_2_4"  "49_2_5"  "49_2_6" 
[246] "49_2_7"  "49_2_8"  "49_2_9"  "49_2_10" "49_2_11" "49_2_12" "53_3_1" 
[253] "53_3_2"  "53_3_3"  "53_3_4"  "53_3_5"  "53_3_6"  "53_3_7"  "53_3_8" 
[260] "53_3_9"  "53_3_10" "53_3_11" "53_3_12" "54_3_1"  "54_3_2"  "54_3_3" 
[267] "54_3_4"  "54_3_5"  "54_3_6"  "54_3_7"  "54_3_8"  "54_3_9"  "54_3_10"
[274] "54_3_11" "54_3_12" "60_1_1"  "60_1_2"  "60_1_3"  "60_1_4"  "60_1_5" 
[281] "60_1_6"  "60_1_7"  "60_1_8"  "60_1_9"  "60_1_10" "60_1_11" "60_1_12"

In [349]:
# Broadcast intercept to each row (sample)
intercept_matrix <- matrix(rep(results[, "(Intercept)"], each=m), nrow=m, byrow=TRUE)

In [350]:
env_matrix <- matrix(X, nrow=m, ncol=n, byrow=FALSE)

In [351]:
dim(intercept_matrix)

[1] 287   7

In [352]:
dim(env_matrix)

[1] 287   7

In [353]:
slopes = results[, "X"]

In [389]:
slopes

[1] -0.011780518 -0.019304551 -0.002941847 -0.029017997  0.030745603
[6] -0.023053532 -0.002290955

In [392]:
effect_matrix

0.0016758409,0.0016758409,0.0016758409,0.0016758409,0.0016758409,0.0016758409,0.0016758409
0.0027461745,0.0027461745,0.0027461745,0.0027461745,0.0027461745,0.0027461745,0.0027461745
0.0004184933,0.0004184933,0.0004184933,0.0004184933,0.0004184933,0.0004184933,0.0004184933
0.0041279635,0.0041279635,0.0041279635,0.0041279635,0.0041279635,0.0041279635,0.0041279635
-0.0043737247,-0.0043737247,-0.0043737247,-0.0043737247,-0.0043737247,-0.0043737247,-0.0043737247
0.0032794868,0.0032794868,0.0032794868,0.0032794868,0.0032794868,0.0032794868,0.0032794868
0.0003259005,0.0003259005,0.0003259005,0.0003259005,0.0003259005,0.0003259005,0.0003259005
0.0016758409,0.0016758409,0.0016758409,0.0016758409,0.0016758409,0.0016758409,0.0016758409
0.0027461745,0.0027461745,0.0027461745,0.0027461745,0.0027461745,0.0027461745,0.0027461745
0.0004184933,0.0004184933,0.0004184933,0.0004184933,0.0004184933,0.0004184933,0.0004184933
0.0041279635,0.0041279635,0.0041279635,0.0041279635,0.0041279635,0.0041279635,0.0041279635


In [354]:
# Multiply env_matrix by slopes (element-wise)
effect_matrix <- env_matrix * slopes

# Add intercept matrix
allele_freq_pred <- intercept_matrix + effect_matrix

In [355]:
correlations <- sapply(1:ncol(Y), function(i) cor(Y[, i],allele_freq_pred[, i]))

In [356]:
dim(matrix(correlations))

[1] 7 1

In [357]:
correlations * 100

[1]   8.822613  13.653612  -1.098335  22.738743 -10.876799   3.412952 -16.655494

In [359]:
effect_size

function (Y, X, lfmm.object) 
{
    if (ncol(X) > 1) 
        stop("Indirect effect sizes are computed for \n                        a single variable (d=1).")
    Y = scale(Y, scale = FALSE)
    X = scale(X, scale = FALSE)
    reg.lm <- function(i) {
        datafr <- data.frame(Y[, i], lfmm.object$U)
        lm(X ~ ., data = datafr)$coefficients[2]
    }
    p <- ncol(Y)
    effect.sizes <- sapply(1:p, FUN = reg.lm)
    return(effect.sizes)
}
<bytecode: 0x558d1d3c65e0>
<environment: namespace:lfmm>

In [196]:
dim(X_new_matrix)

[1]   287 13985

In [195]:
# Assuming X is your environmental matrix and results contain intercepts and slopes
X_new_matrix <- matrix(rep(X, each = ncol(Y)), nrow = length(X), ncol = ncol(Y))

# Calculation of predictions seems to be incorrect due to dimension mismatch; revise it:
predictions <- matrix(results[, 1], nrow = nrow(X_new_matrix), ncol = ncol(X_new_matrix), byrow = TRUE) + X_new_matrix * matrix(results[, 2], nrow = nrow(X_new_matrix), ncol = ncol(X_new_matrix), byrow = TRUE)

# Calculating correlations
#correlations <- sapply(1:ncol(Y), function(i) cor(Y[, i], predictions[, i]))


In [197]:
predictions <- results[, "(Intercept)"] + X_new_matrix * results[, "X"]

In [193]:
## old 
X_new_matrix <- matrix(rep(X, each = length(results[, "X"])), nrow = length(X), ncol = length(results[, "X"]))

# Perform matrix multiplication
predictions <- results[, "(Intercept)"] + X_new_matrix * results[, "X"]


correlations <- sapply(1:ncol(Y), function(i) cor(Y[, i], predictions[, i]))

In [199]:
correlations <- sapply(1:ncol(Y), function(i) cor(Y[, i], predictions[, i]))

In [200]:
correlations

[1]  3.306867e-02 -7.771422e-02  1.177513e-02  1.362275e-01  2.995176e-02
    [6]  1.233473e-01 -5.625246e-02 -1.644504e-01  2.615450e-02  4.687460e-02
   [11]  3.351595e-02 -1.463627e-01  7.562399e-03 -2.579703e-02  6.409167e-04
   [16] -7.545038e-02 -1.238356e-02 -1.700895e-02  3.540773e-02 -1.631677e-02
   [21]  4.624317e-02  2.039184e-02 -3.778607e-02  8.131628e-02 -5.389444e-02
   [26]  4.401086e-02  6.449503e-02 -5.576882e-02 -4.664915e-02 -3.551821e-02
   [31]  2.939757e-02 -8.161442e-02 -2.572571e-02 -8.858379e-02 -2.019712e-02
   [36]  3.571278e-02  7.149003e-02  3.748162e-02 -1.395720e-02  3.380397e-02
   [41]  1.478169e-03 -1.279815e-02 -4.320971e-02 -6.122314e-02  3.862942e-02
   [46] -1.562620e-02  2.637727e-02  1.532139e-02  2.884582e-02 -1.070997e-02
   [51] -1.029528e-01  1.143374e-02  1.313309e-02  1.274373e-01  4.936142e-02
   [56]  2.945672e-02 -2.428563e-02 -1.690730e-02  1.238375e-01 -5.616249e-02
   [61] -2.184190e-02  5.797338e-02  2.363008e-02  1.899518e-02  9.896994e-02
   [66] -7.485326e-03  3.530903e-02 -1.745835e-01 -5.910623e-03  7.228313e-02
   [71] -9.955271e-03  1.058247e-02  5.781963e-02  3.625333e-02 -1.065375e-01
   [76] -1.052514e-02  5.844513e-02 -9.544002e-03 -5.988691e-02 -4.932641e-02
   [81] -5.218830e-02 -3.543731e-02 -2.429955e-03 -2.663004e-02  5.350429e-02
   [86] -4.265504e-04  5.164096e-02 -2.854917e-02 -9.550607e-02  8.292199e-02
   [91] -3.641116e-02 -2.965056e-02 -2.849008e-02  3.809673e-03 -3.825885e-02
   [96] -8.152066e-02  2.580347e-02 -3.977049e-02  3.209493e-02  5.223064e-02
  [101] -1.123825e-01 -1.911160e-02  6.450383e-02 -5.570897e-02 -7.213607e-03
  [106]  3.251122e-02 -5.280746e-02 -1.057107e-02  5.556627e-02  1.916146e-02
  [111]  1.129737e-02  5.806692e-02 -5.645896e-02 -7.659481e-02  6.962174e-02
  [116] -3.036812e-02  4.175196e-02  2.108716e-02 -8.847942e-03  2.437467e-03
  [121]  9.392280e-03 -2.334849e-02 -3.904741e-02 -4.043489e-02 -1.146140e-02
  [126]  6.489214e-02 -3.864591e-03  7.829118e-03  6.420934e-02 -7.913872e-02
  [131]  6.097353e-02  7.418370e-02 -4.611984e-02  2.278540e-02 -1.282615e-01
  [136] -3.183869e-03 -8.449076e-03  3.949447e-02 -8.525470e-03 -3.711507e-02
  [141]  1.045085e-02 -1.738007e-02  4.962330e-02  1.984710e-02 -5.612030e-02
  [146]  8.724995e-02 -6.096133e-02  2.504137e-02  7.802813e-03 -3.496630e-02
  [151]  1.698818e-02 -7.651260e-02 -7.515414e-02  7.656252e-02  4.952544e-02
  [156]  4.861625e-02  3.545664e-02 -8.583833e-03  1.410441e-01 -6.796705e-02
  [161]  4.697857e-02 -5.644474e-02  3.558103e-02 -6.277299e-02 -2.069004e-02
  [166] -1.205994e-02 -1.622432e-02 -6.836750e-02 -4.440964e-02 -6.401355e-02
  [171] -1.511424e-02 -1.308340e-01  4.059435e-02  1.820379e-01  3.336604e-02
  [176]  7.647054e-02  9.142621e-03  1.431283e-01 -3.125649e-02 -2.313491e-02
  [181] -4.478143e-02 -9.977022e-02 -6.614532e-02 -2.315766e-02  1.494539e-01
  [186] -3.057252e-02  1.065937e-01 -2.543650e-02  1.857242e-02  3.254416e-02
  [191] -1.103830e-02 -1.919885e-02  3.046841e-02 -6.393582e-02 -7.159707e-03
  [196] -7.918220e-02  1.067814e-02 -1.708231e-03  3.857887e-02  1.128515e-01
  [201] -2.204263e-02 -1.282779e-02  4.533334e-02 -3.238452e-02 -7.453702e-02
  [206]  3.018232e-02  6.714467e-02  1.059408e-01  2.812686e-03 -4.095089e-02
  [211] -9.086863e-03  6.030078e-02  8.905832e-02  7.814376e-02  4.524409e-02
  [216]  7.692086e-02 -6.316814e-02 -6.392200e-02 -1.291386e-02 -6.518479e-02
  [221] -2.005149e-02 -1.857635e-02 -9.526527e-02 -3.148345e-02  2.164067e-02
  [226]  4.770366e-02 -8.037036e-02  3.501743e-02  5.024732e-02  4.251586e-02
  [231] -1.683642e-02 -1.210972e-01  7.494667e-02 -2.821870e-02 -1.281801e-02
  [236]  2.601041e-02  8.669082e-02 -5.154783e-02 -2.669123e-02  3.694407e-02
  [241]  9.545306e-03  3.845384e-02  1.133175e-01  1.107097e-01 -4.045160e-02
  [246]  2.859446e-02  8.369429e-02 -7.221779e-02 -2.937399e-02  3.317066e-03
  [251]  4.968622e-02  1.872573e-02  2.799189e-02  4.031168e-02 -3.795963e-02
  [256] -4.519990e-02 -1.0

In [144]:
env_test <- read.csv(sprintf('%s/testing_environment_lea_full_genome.csv', results_dir), header = TRUE, colClasses = "numeric")
rownames(env_test) <- NULL
colnames(env_test) <- NULL
X_new <- as.matrix(env_test)

In [145]:
X_new = as.matrix(X_new[,3])

In [156]:
dim(as.matrix(results[, "X"]))

[1] 13985     1

In [172]:
dim(Y)

[1]   287 13985

In [ ]:
X_new_matrix <- matrix(rep(X, each = length(results[, "X"])), nrow = length(X), ncol = length(results[, "X"]))

# Perform matrix multiplication
predictions <- results[, "(Intercept)"] + X_new_matrix * results[, "X"]

In [ ]:
dim(predictions) # 287 * 13985


dim(Y) # 287 * 13985

correlations <- apply(1:ncol(Y), 1, function(i) cor(Y[, i], predictions[, i]))


Error in apply(1:ncol(Y), 1, function(i) cor(Y[, i], predictions[, i])): dim(X) must have a positive length
Traceback:

1. apply(1:ncol(Y), 1, function(i) cor(Y[, i], predictions[, i]))
2. stop("dim(X) must have a positive length")

In [ ]:
predictions <- scale(predictions, scale = FALSE)

In [187]:
# Calculating correlation for each SNP directly on matrices
correlations <- sapply(1:ncol(Y), function(i) cor(Y[, i], predictions[, i]))


In [188]:
correlations

[1]  3.306867e-02 -7.771422e-02  1.177513e-02  1.362275e-01  2.995176e-02
    [6]  1.233473e-01 -5.625246e-02 -1.644504e-01  2.615450e-02  4.687460e-02
   [11]  3.351595e-02 -1.463627e-01  7.562399e-03 -2.579703e-02  6.409167e-04
   [16] -7.545038e-02 -1.238356e-02 -1.700895e-02  3.540773e-02 -1.631677e-02
   [21]  4.624317e-02  2.039184e-02 -3.778607e-02  8.131628e-02 -5.389444e-02
   [26]  4.401086e-02  6.449503e-02 -5.576882e-02 -4.664915e-02 -3.551821e-02
   [31]  2.939757e-02 -8.161442e-02 -2.572571e-02 -8.858379e-02 -2.019712e-02
   [36]  3.571278e-02  7.149003e-02  3.748162e-02 -1.395720e-02  3.380397e-02
   [41]  1.478169e-03 -1.279815e-02 -4.320971e-02 -6.122314e-02  3.862942e-02
   [46] -1.562620e-02  2.637727e-02  1.532139e-02  2.884582e-02 -1.070997e-02
   [51] -1.029528e-01  1.143374e-02  1.313309e-02  1.274373e-01  4.936142e-02
   [56]  2.945672e-02 -2.428563e-02 -1.690730e-02  1.238375e-01 -5.616249e-02
   [61] -2.184190e-02  5.797338e-02  2.363008e-02  1.899518e-02  9.896994e-02
   [66] -7.485326e-03  3.530903e-02 -1.745835e-01 -5.910623e-03  7.228313e-02
   [71] -9.955271e-03  1.058247e-02  5.781963e-02  3.625333e-02 -1.065375e-01
   [76] -1.052514e-02  5.844513e-02 -9.544002e-03 -5.988691e-02 -4.932641e-02
   [81] -5.218830e-02 -3.543731e-02 -2.429955e-03 -2.663004e-02  5.350429e-02
   [86] -4.265504e-04  5.164096e-02 -2.854917e-02 -9.550607e-02  8.292199e-02
   [91] -3.641116e-02 -2.965056e-02 -2.849008e-02  3.809673e-03 -3.825885e-02
   [96] -8.152066e-02  2.580347e-02 -3.977049e-02  3.209493e-02  5.223064e-02
  [101] -1.123825e-01 -1.911160e-02  6.450383e-02 -5.570897e-02 -7.213607e-03
  [106]  3.251122e-02 -5.280746e-02 -1.057107e-02  5.556627e-02  1.916146e-02
  [111]  1.129737e-02  5.806692e-02 -5.645896e-02 -7.659481e-02  6.962174e-02
  [116] -3.036812e-02  4.175196e-02  2.108716e-02 -8.847942e-03  2.437467e-03
  [121]  9.392280e-03 -2.334849e-02 -3.904741e-02 -4.043489e-02 -1.146140e-02
  [126]  6.489214e-02 -3.864591e-03  7.829118e-03  6.420934e-02 -7.913872e-02
  [131]  6.097353e-02  7.418370e-02 -4.611984e-02  2.278540e-02 -1.282615e-01
  [136] -3.183869e-03 -8.449076e-03  3.949447e-02 -8.525470e-03 -3.711507e-02
  [141]  1.045085e-02 -1.738007e-02  4.962330e-02  1.984710e-02 -5.612030e-02
  [146]  8.724995e-02 -6.096133e-02  2.504137e-02  7.802813e-03 -3.496630e-02
  [151]  1.698818e-02 -7.651260e-02 -7.515414e-02  7.656252e-02  4.952544e-02
  [156]  4.861625e-02  3.545664e-02 -8.583833e-03  1.410441e-01 -6.796705e-02
  [161]  4.697857e-02 -5.644474e-02  3.558103e-02 -6.277299e-02 -2.069004e-02
  [166] -1.205994e-02 -1.622432e-02 -6.836750e-02 -4.440964e-02 -6.401355e-02
  [171] -1.511424e-02 -1.308340e-01  4.059435e-02  1.820379e-01  3.336604e-02
  [176]  7.647054e-02  9.142621e-03  1.431283e-01 -3.125649e-02 -2.313491e-02
  [181] -4.478143e-02 -9.977022e-02 -6.614532e-02 -2.315766e-02  1.494539e-01
  [186] -3.057252e-02  1.065937e-01 -2.543650e-02  1.857242e-02  3.254416e-02
  [191] -1.103830e-02 -1.919885e-02  3.046841e-02 -6.393582e-02 -7.159707e-03
  [196] -7.918220e-02  1.067814e-02 -1.708231e-03  3.857887e-02  1.128515e-01
  [201] -2.204263e-02 -1.282779e-02  4.533334e-02 -3.238452e-02 -7.453702e-02
  [206]  3.018232e-02  6.714467e-02  1.059408e-01  2.812686e-03 -4.095089e-02
  [211] -9.086863e-03  6.030078e-02  8.905832e-02  7.814376e-02  4.524409e-02
  [216]  7.692086e-02 -6.316814e-02 -6.392200e-02 -1.291386e-02 -6.518479e-02
  [221] -2.005149e-02 -1.857635e-02 -9.526527e-02 -3.148345e-02  2.164067e-02
  [226]  4.770366e-02 -8.037036e-02  3.501743e-02  5.024732e-02  4.251586e-02
  [231] -1.683642e-02 -1.210972e-01  7.494667e-02 -2.821870e-02 -1.281801e-02
  [236]  2.601041e-02  8.669082e-02 -5.154783e-02 -2.669123e-02  3.694407e-02
  [241]  9.545306e-03  3.845384e-02  1.133175e-01  1.107097e-01 -4.045160e-02
  [246]  2.859446e-02  8.369429e-02 -7.221779e-02 -2.937399e-02  3.317066e-03
  [251]  4.968622e-02  1.872573e-02  2.799189e-02  4.031168e-02 -3.795963e-02
  [256] -4.519990e-02 -1.0

In [173]:
correlations <- apply(Y, 2, function(y) cor(y, predictions[, which(colnames(predictions) == colnames(Y))]))

In [175]:
print(correlations)

numeric(0)


In [124]:
#results[, "intercept.(Intercept)"]

In [ ]:
results[, "slope"]

In [ ]:
predictions <- results[, "intercept"] + new_X * results[, "slope"] + 

In [101]:
datafr

ERROR: Error in eval(expr, envir, enclos): object 'datafr' not found


In [97]:
Y = allele_freq
X = as.matrix(env[,2])

Y = scale(Y, scale = FALSE)
X = scale(X, scale = FALSE)

# Function to compute regression coefficients of Y on X
reg.lm <- function(i) {
    datafr <- data.frame(X[, i], mod.lfmm$U)  # Assuming mod.lfmm$U are other predictors
    lm(Y ~ ., data = datafr)$coefficients[2]  # Get the coefficient for X
}

p <- ncol(X)  # Assuming you want to run this for each column in X
effect.sizes <- sapply(1:p, FUN = reg.lm)

In [98]:
dim(as.matrix(effect.sizes))

[1] 1 1

In [92]:
Y = allele_freq

X = as.matrix(env[,2])

In [93]:
# Scaling the matrix X and vector Y
Y = scale(Y, scale = FALSE)  # Center Y
X = scale(X, scale = FALSE)  # Center X by each SNP

# Function to compute regression coefficients of Y on each SNP in X
reg.lm <- function(i) {
    datafr <- data.frame(Y = Y, SNP = X[, i])
    lm(Y ~ SNP, data = datafr)$coefficients[2]  # Retrieve the coefficient for the SNP
}

p <- ncol(X)  # Number of SNPs
effect.sizes <- sapply(1:p, FUN = reg.lm)  # Apply the function to each SNP

In [94]:
effect.sizes

[1] -0.002882316

In [80]:
effect.sizes

[1] 0.001448962

In [36]:
env_new <- read.csv(sprintf('%s/testing_environment_lea_full_genome.csv', results_dir), header = TRUE, colClasses = "numeric")


In [40]:
as.matrix(env_new[,'bio1'])

0.8744907
-2.0040338
-1.3790421
-0.5036921
-0.6139786
-0.4358370


In [ ]:
testing_environment_lea_full_genome.csv

In [42]:
predicted = predict_lfmm(allele_freq, as.matrix(env[,1]), mod.lfmm, fdr.level = 0.1, newdata = as.matrix(env_new[,'bio1']))

In [ ]:
predicted

In [53]:
b.values <- effect_size(Y = allele_freq, X = as.matrix(env[,2]), lfmm.object = mod.lfmm) 

In [54]:
b.values

Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 7.971142e-01 -2.770718e-01 -1.766795e+00 -1.006880e+00  3.491161e-02 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 4.432351e-01 -1.802381e+00  1.301478e+00 -7.552083e-01  8.473013e-01 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.851511e+00 -1.457407e+00 -4.787709e-01  5.000226e-01  1.069100e-02 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.461747e+00 -7.776567e-01 -1.304248e+00  5.494755e-01 -1.074136e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 5.110917e-02  6.405958e-01  1.066060e+00 -9.380765e-01  8.166272e-02 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 4.124716e-01 -8.380442e-01 -1.892038e-01 -8.669326e-01  1.510402e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 7.923011e-01  2.705435e+00  3.811838e-01 -1.993578e+00  1.284441e-01 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 3.361413e-01  4.051836e-01 -1.046042e+00  1.179400e+00  1.380921e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.103975e+00  4.249339e-01  7.872302e-02  2.871258e-01 -1.160041e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-2.036434e+00  3.435541e+00 -2.462504e-01  6.577947e-01 -1.262955e-02 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-6.904440e-01  1.056456e+00 -7.017513e+00 -6.968868e-02  1.095567e-01 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.183887e-01  1.141916e+00  2.387016e+00  4.579060e-01 -7.561564e-01 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 7.013048e-01  6.304148e-01  2.108586e-01 -1.016078e+00  5.572380e-01 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.048357e+00 -2.118188e-01  5.334824e+00  1.039390e+00 -1.259688e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.768699e+00 -4.557796e-01 -9.627332e-01  1.473958e+00  8.243429e-01 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-4.966476e-01 -2.504638e-01 -3.655737e-01  9.129584e-01  1.094020e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-2.080944e+00 -9.117116e-01 -8.136524e-01 -1.438288e-01 -1.618480e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 7.386655e-01 -1.777702e-01 -2.879276e-01  3.910530e-01 -5.527206e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.739997e+00 -1.739517e+00  6.313573e-01  2.792572e-01  1.424316e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.088473e+00  1.348781e+00 -8.703653e-02 -2.113769e+00 -1.369386e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 5.279517e-01 -8.730002e-01 -2.905796e+00 -2.275299e+00 -1.319446e-01 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.110690e+00  3.890154e-01  1.925361e-01 -9.131968e-01 -1.060363e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.107948e+00 -4.677645e-02  1.943918e+00 -1.685549e-01 -5.098258e-01 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 9.831895e-01  3.708777e-01  7.611016e-01  1.704763e+00 -1.161862e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-3.546803e-01  8.161204e-01  1.416910e+00 -7.312423e-01  1.981376e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-8.001017e-01 -5.959099e-01 -7.620873e-01  6.622797e-01  7.784729e-01 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 4.487834e-02  1.217081e-01 -2.056442e+00  1.034209e+00 -3.116460e+00 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.590313e+00  5.335662e-01  1.504788e+00  2.192268e-01  1.488174e+00 
       Y...i.        Y...i.    

In [48]:
b.values <- effect_size(Y = allele_freq, X = as.matrix(env[,1]), lfmm.object = mod.lfmm) 

In [49]:
b.values

Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.8108724659  1.5274495585  6.1073908368  1.0239759026  0.9463381324 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 2.5345089459  3.2618601445  0.2531971651 -0.4263842228 -2.1296765886 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.0340039497 -0.1344660778  0.1527584172 -0.2103163423 -0.2638402607 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.2512936486  3.0044716168  1.3459729166 -0.7838209845  1.3090758472 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 0.3444650261  0.2744250022 -0.0206181225  0.6406122204 -0.3360885172 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.5431166985  1.1298493706  0.9464516665  0.5901303959 -0.1720736150 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 0.7029670703 -1.3440112089 -0.4022749292  0.4912713890  0.2027630660 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 0.3112369445 -0.9316775475  1.6013819964  1.7587624673 -0.0989034116 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.0794249791  0.3118897310  0.7645515609 -2.0699870389  2.1603629759 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.6082728225 -3.1075054440 -2.8055263070 -0.6257254414  0.4421692244 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 2.6601913029 -0.0258523813  2.6342767702  1.0184559129 -0.2474122429 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.0710455657  0.0981057660 -1.4528903951 -0.4923052023 -1.3930824058 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-0.8700625326  0.3176005001  1.2984642091  0.7169151624 -0.4566882140 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-0.8015601920  0.7063244331 -3.7944716554  0.5441422130  0.0898468961 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-0.2463767247  1.1671763881  0.5273611426  0.4862359828  0.6335425207 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.7587118710  0.9141106048  0.1399917632  1.4375377626 -0.2969834492 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.5520514645  1.3007573724 -0.4656342239 -0.9313901545  1.7901366628 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 0.2812861363 -0.2525110428 -0.5905837151  0.6682913954  5.0086345526 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.1679541846  1.4062898012 -1.3237984171 -0.1674905814 -0.7049127610 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 0.2558341342 -2.6686670018 -1.2507046463  0.5759024541  2.3180242046 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 1.0288014775  2.1866385524  2.5552192665 -0.0719540844  0.9190471248 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.1481526134 -1.1860893572 -1.0802366944  0.5403414842  1.0779694369 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-0.9094150765  0.2719711116 -2.1734231517 -0.4195468793  1.0200833519 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-0.2761234308 -0.9458323374 -1.7838720697 -1.1103892030 -0.0163964513 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-1.0735112488  0.8430981980 -0.9904173180  0.6665774454 -1.7137067006 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 0.3367922725 -1.0133754321  0.8754316613  0.4204192777 -0.1295072138 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
-0.3904748824 -1.1159179867 -0.1156143310  0.1965672648 -1.7034284318 
       Y...i.        Y...i.        Y...i.        Y...i.        Y...i. 
 0.9327306079 -0.3875337897  0.0017084256 -0.4898699348 -3.8651688241 
       Y...i.        Y...i.    

In [ ]:
x.pred <- scale(y[,candidates], scale = F)%*% matrix(b.values[candidates])

In [ ]:
pv$

In [ ]:
mod2 <- lfmm2(input = allele_freq, env = env, K = k)
pv <- lfmm2.test(object = mod2, input = allele_freq, env = env)

p_values <- pv$pvalues
zscores <- pv$zscores
gif <- pv$gif

write.csv(p_values, file = output_file_pvalue)
write.csv(zscores, file = output_file_zscores)
write.csv(gif, file = output_file_gif)

##calculate qvalues 
## gif has already been conducted 
p_values[, 1] <- NULL 
qvalues <- qvalue(p_values)$qvalue
write.csv(qvalues, file = output_file_qvalue)

In [20]:
## Simulation of 1000 genotypes for 100 individuals (y)
u <- matrix(rnorm(300, sd = 1), nrow = 100, ncol = 2)
v <- matrix(rnorm(3000, sd = 2), nrow = 2, ncol = 1000)
y <- matrix(rbinom(100000, size = 2,
                  prob = 1/(1 + exp(-0.3*(u%*%v
                  + rnorm(100000, sd = 2))))),
                  nrow = 100,
                  ncol = 1000)

Warning message in matrix(rnorm(300, sd = 1), nrow = 100, ncol = 2):
“data length differs from size of matrix: [300 != 100 x 2]”
Warning message in matrix(rnorm(3000, sd = 2), nrow = 2, ncol = 1000):
“data length differs from size of matrix: [3000 != 2 x 1000]”


In [21]:
## Simulation of 1000 phenotypes (x)
## Only the last 10 genotypes have significant effect sizes (b)
b <- matrix(c(rep(0, 990), rep(6000, 10)))
x <- y%*%b + rnorm(100, sd = 100)

## Compute effect sizes using lfmm_ridge
## Note that centering is important (scale = F).
mod.lfmm <- lfmm_ridge(Y = y,
                  X = x,
                  K = 2)

## Compute direct effect sizes using lfmm_ridge estimates
b.estimates <- effect_size(y, x, mod.lfmm)

In [23]:
x

36002.60
66018.51
59854.46
42162.37
42051.52
47822.11
71876.35
72120.10
59800.80
48137.90
60078.02


In [22]:
y

2,1,0,0,0,1,1,1,2,1,⋯,0,0,0,2,0,1,1,0,0,2
2,1,1,0,2,2,1,1,0,0,⋯,0,2,1,0,1,2,1,2,0,2
1,1,1,0,0,1,1,0,0,1,⋯,0,2,0,0,1,2,2,2,0,1
2,1,0,1,0,0,0,2,2,2,⋯,0,0,1,1,2,0,0,2,0,1
2,2,2,2,1,2,1,1,1,0,⋯,0,0,0,1,0,1,0,2,1,2
1,2,0,0,2,1,2,2,0,1,⋯,0,0,0,2,1,1,0,0,2,2
1,1,1,1,2,1,0,1,1,2,⋯,2,0,1,2,2,1,2,0,1,1
0,2,2,2,2,2,2,1,1,0,⋯,0,2,2,0,2,2,1,1,1,1
0,1,1,0,2,1,1,0,2,1,⋯,1,0,0,1,2,2,1,1,2,0
1,0,1,0,0,1,1,1,2,2,⋯,1,1,1,1,1,0,1,1,1,0
2,1,1,1,0,0,0,0,0,0,⋯,1,0,1,2,1,0,2,1,2,0


In [10]:
train_samples_vector = c('1_2_2',
  '1_2_3',
  '1_3_1',
  '1_3_4',
  '1_3_5',
  '1_3_6',
  '1_3_7',
  '1_3_8',
  '1_3_9',
  '1_3_10',
  '1_3_11',
  '1_3_12',
  '2_1_8',
  '2_1_9',
  '2_1_11',
  '2_1_12',
  '2_2_5',
  '2_2_6',
  '2_2_7',
  '5_3_1',
  '5_3_2',
  '5_3_3',
  '5_3_4',
  '5_3_5',
  '5_3_6',
  '5_3_7',
  '5_3_8',
  '5_3_9',
  '5_3_10',
  '5_3_11',
  '5_3_12',
  '6_1_1',
  '6_1_4',
  '6_2_2',
  '6_2_3',
  '6_2_5',
  '6_2_6',
  '6_2_7',
  '6_2_8',
  '6_2_9',
  '6_2_10',
  '6_2_11',
  '6_2_12',
  '9_3_1',
  '9_3_2',
  '9_3_3',
  '9_3_4',
  '9_3_5',
  '9_3_6',
  '9_3_7',
  '9_3_8',
  '9_3_9',
  '9_3_10',
  '9_3_11',
  '9_3_12',
  '10_1_3',
  '10_1_4',
  '10_1_6',
  '10_1_7',
  '10_1_11',
  '10_1_12',
  '10_2_1',
  '10_2_5',
  '10_2_8',
  '10_2_9',
  '10_3_2',
  '10_3_10',
  '11_1_4',
  '11_1_8',
  '11_1_9',
  '11_1_11',
  '11_2_2',
  '11_2_7',
  '11_2_12',
  '11_3_1',
  '11_3_3',
  '11_3_5',
  '11_3_6',
  '12_1_2',
  '12_1_3',
  '12_1_5',
  '12_1_6',
  '12_1_7',
  '12_1_8',
  '12_1_9',
  '12_2_4',
  '12_2_10',
  '12_2_11',
  '12_2_12',
  '13_1_6',
  '13_2_7',
  '13_3_1',
  '13_3_2',
  '13_3_3',
  '13_3_4',
  '13_3_5',
  '13_3_8',
  '13_3_9',
  '13_3_10',
  '13_3_11',
  '13_3_12',
  '23_1_1',
  '23_1_2',
  '23_1_3',
  '23_1_4',
  '23_1_5',
  '23_1_6',
  '23_1_7',
  '23_1_8',
  '23_1_9',
  '23_1_10',
  '23_1_11',
  '23_1_12',
  '25_2_1',
  '25_2_2',
  '25_2_3',
  '25_2_4',
  '25_2_5',
  '25_2_6',
  '25_2_7',
  '25_2_8',
  '25_2_9',
  '25_2_10',
  '25_2_11',
  '25_2_12',
  '26_1_1',
  '26_1_2',
  '26_1_3',
  '26_1_4',
  '26_1_5',
  '26_1_6',
  '26_1_8',
  '26_1_9',
  '26_1_10',
  '26_1_11',
  '26_1_12',
  '27_2_2',
  '27_2_3',
  '27_2_5',
  '27_3_1',
  '27_3_4',
  '27_3_6',
  '27_3_7',
  '27_3_8',
  '27_3_9',
  '27_3_10',
  '27_3_11',
  '27_3_12',
  '28_2_7',
  '28_2_8',
  '28_2_11',
  '28_3_2',
  '28_3_3',
  '28_3_4',
  '28_3_6',
  '28_3_12',
  '28_3_13',
  '28_3_15',
  '28_3_16',
  '28_3_17',
  '32_1_5',
  '32_1_11',
  '32_2_8',
  '32_2_9',
  '32_3_1',
  '32_3_2',
  '32_3_3',
  '32_3_4',
  '32_3_6',
  '32_3_7',
  '32_3_10',
  '32_3_12',
  '33_1_1',
  '33_2_4',
  '33_2_5',
  '33_2_6',
  '33_2_9',
  '33_2_11',
  '33_2_12',
  '37_1_1',
  '37_1_2',
  '37_1_3',
  '37_1_4',
  '37_1_5',
  '37_1_6',
  '37_1_7',
  '37_1_8',
  '37_1_9',
  '37_1_10',
  '37_1_11',
  '37_1_12',
  '42_1_2',
  '42_1_12',
  '42_2_11',
  '42_3_1',
  '42_3_3',
  '42_3_4',
  '42_3_5',
  '42_3_6',
  '42_3_7',
  '42_3_8',
  '42_3_9',
  '42_3_10',
  '43_1_3',
  '43_1_10',
  '43_2_1',
  '43_3_2',
  '43_3_4',
  '43_3_5',
  '43_3_6',
  '43_3_7',
  '43_3_8',
  '43_3_9',
  '43_3_11',
  '43_3_12',
  '45_2_3',
  '45_2_7',
  '45_3_1',
  '45_3_2',
  '45_3_4',
  '45_3_5',
  '45_3_6',
  '45_3_8',
  '45_3_9',
  '45_3_10',
  '45_3_11',
  '45_3_12',
  '46_3_1',
  '46_3_2',
  '46_3_3',
  '46_3_4',
  '46_3_5',
  '46_3_6',
  '46_3_7',
  '46_3_8',
  '46_3_9',
  '46_3_10',
  '46_3_11',
  '46_3_12',
  '49_2_1',
  '49_2_2',
  '49_2_3',
  '49_2_4',
  '49_2_5',
  '49_2_6',
  '49_2_7',
  '49_2_8',
  '49_2_9',
  '49_2_10',
  '49_2_11',
  '49_2_12',
  '53_3_1',
  '53_3_2',
  '53_3_3',
  '53_3_4',
  '53_3_5',
  '53_3_6',
  '53_3_7',
  '53_3_8',
  '53_3_9',
  '53_3_10',
  '53_3_11',
  '53_3_12',
  '54_3_1',
  '54_3_2',
  '54_3_3',
  '54_3_4',
  '54_3_5',
  '54_3_6',
  '54_3_7',
  '54_3_8',
  '54_3_9',
  '54_3_10',
  '54_3_11',
  '54_3_12',
  '60_1_1',
  '60_1_2',
  '60_1_3',
  '60_1_4',
  '60_1_5',
  '60_1_6',
  '60_1_7',
  '60_1_8',
  '60_1_9',
  '60_1_10',
  '60_1_11',
  '60_1_12')

# 